In [1]:
%%capture
!pip install "flair" -q
!pip install "scispacy" -q
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz -q

LSTM-CRF : [link](https://github.com/flairNLP/flair/blob/master/resources/docs/HUNFLAIR_TUTORIAL_2_TRAINING.md)

Transformer: [link](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_TRAINING_SEQUENCE_LABELER.md)

In [2]:
import flair
import transformers
import torch

In [3]:
flair.__version__, transformers.__version__, torch.__version__

('0.12.1', '4.25.1', '1.13.1+cu116')

In [4]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.embeddings import TransformerWordEmbeddings

In [5]:
from flair.datasets import NCBI_DISEASE

In [6]:
# 1. load your target corpus
corpus = NCBI_DISEASE()

2023-03-21 13:32:25,327 https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/NCBItrainset_corpus.zip not found in cache, downloading to /tmp/tmp7qa5neih


100%|██████████| 317k/317k [00:00<00:00, 3.08MB/s]

2023-03-21 13:32:25,575 copying /tmp/tmp7qa5neih to cache at /root/.flair/datasets/ncbi_disease/original/NCBItrainset_corpus.zip
2023-03-21 13:32:25,577 removing temp file /tmp/tmp7qa5neih


2023-03-21 13:32:25,729 https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/NCBIdevelopset_corpus.zip not found in cache, downloading to /tmp/tmpch0e1f6u


100%|██████████| 56.3k/56.3k [00:00<00:00, 1.08MB/s]

2023-03-21 13:32:25,916 copying /tmp/tmpch0e1f6u to cache at /root/.flair/datasets/ncbi_disease/original/NCBIdevelopset_corpus.zip
2023-03-21 13:32:25,918 removing temp file /tmp/tmpch0e1f6u
2023-03-21 13:32:26,055 https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/NCBItestset_corpus.zip not found in cache, downloading to /tmp/tmp9e31tbp9



100%|██████████| 58.8k/58.8k [00:00<00:00, 1.18MB/s]

2023-03-21 13:32:26,245 copying /tmp/tmp9e31tbp9 to cache at /root/.flair/datasets/ncbi_disease/original/NCBItestset_corpus.zip
2023-03-21 13:32:26,247 removing temp file /tmp/tmp9e31tbp9



Converting to CoNLL: 100%|██████████| 100/100 [00:06<00:00, 15.49it/s]

2023-03-21 13:33:23,667 Reading data from /root/.flair/datasets/ncbi_disease
2023-03-21 13:33:23,669 Train: /root/.flair/datasets/ncbi_disease/SciSpacySentenceSplitter_core_sci_sm_0.5.1_SciSpacyTokenizer_core_sci_sm_0.5.1_train.conll
2023-03-21 13:33:23,671 Dev: /root/.flair/datasets/ncbi_disease/SciSpacySentenceSplitter_core_sci_sm_0.5.1_SciSpacyTokenizer_core_sci_sm_0.5.1_dev.conll
2023-03-21 13:33:23,673 Test: /root/.flair/datasets/ncbi_disease/SciSpacySentenceSplitter_core_sci_sm_0.5.1_SciSpacyTokenizer_core_sci_sm_0.5.1_test.conll


In [7]:
sen = list(corpus.train)[0]
sen

Sentence[15]: "A common human skin tumour is caused by activating mutations in beta-catenin." → ["skin tumour"/Disease]

In [10]:
# 2. load the pre-trained sequence tagger
tagger = SequenceTagger.load("hunflair-disease")
# Making it compatible with the legacy format
tagger.embeddings.embeddings[0].embedding = tagger.embeddings.embeddings[0].embedding
# 3. initialize trainer
trainer = ModelTrainer(tagger, corpus)

2023-03-21 13:35:21,395 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, B-Disease, E-Disease, I-Disease, S-Disease, <START>, <STOP>


In [11]:
# 4. fine-tune on the target corpus
trainer.fine_tune(
    base_path="taggers/lstm-crf",
    train_with_dev=False,
    max_epochs=2,
    learning_rate=0.1,
    mini_batch_size=64
)

2023-03-21 13:35:33,170 ----------------------------------------------------------------------------------------------------
2023-03-21 13:35:33,172 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'pubmed_pmc_wiki_sg_1M.gensim'
      (embedding): Embedding(1000001, 200)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4296

100%|██████████| 15/15 [00:12<00:00,  1.20it/s]

2023-03-21 13:36:51,176 Evaluating as a multi-label problem: False
2023-03-21 13:36:51,194 DEV : loss 0.12372838705778122 - f1-score (micro avg)  0.2927
2023-03-21 13:36:51,213 ----------------------------------------------------------------------------------------------------


2023-03-21 13:36:57,225 epoch 2 - iter 8/84 - loss 0.27587349 - time (sec): 6.01 - samples/sec: 2025.60 - lr: 0.050000
2023-03-21 13:37:03,262 epoch 2 - iter 16/84 - loss 0.28341002 - time (sec): 12.05 - samples/sec: 2099.64 - lr: 0.044737
2023-03-21 13:37:10,856 epoch 2 - iter 24/84 - loss 0.29292839 - time (sec): 19.64 - samples/sec: 1949.10 - lr: 0.039474
2023-03-21 13:37:16,444 epoch 2 - iter 32/84 - loss 0.27916544 - time (sec): 25.23 - samples/sec: 2018.22 - lr: 0.034211
2023-03-21 13:37:22,710 epoch 2 - iter 40/84 - loss 0.27959477 - time (sec): 31.50 - samples/sec: 2028.74 - lr: 0.028947
2023-03-21 13:37:29,404 epoch 2 - iter 48/84 - loss 0.27282602 - time (sec): 38.19 - samples/sec: 2012.80 - lr: 0.023684
2023-03-21 13:37:35,309 epoch 2 - iter 56/84 - loss 0.26254061 - time (sec): 44.09 - samples/sec: 2030.22 - lr: 0.018421
2023-03-21 13:37:41,011 epoch 2 - iter 64/84 - loss 0.25422891 - time (sec): 49.80 - samples/sec: 2048.09 - lr: 0.013158
2023-03-21 13:37:47,194 epoch 2 - 

100%|██████████| 15/15 [00:14<00:00,  1.04it/s]

2023-03-21 13:38:11,808 Evaluating as a multi-label problem: False
2023-03-21 13:38:11,825 DEV : loss 0.08702316880226135 - f1-score (micro avg)  0.4145


2023-03-21 13:38:17,223 ----------------------------------------------------------------------------------------------------
2023-03-21 13:38:17,227 Testing using last state of model ...


100%|██████████| 15/15 [00:12<00:00,  1.22it/s]

2023-03-21 13:38:29,507 Evaluating as a multi-label problem: False
2023-03-21 13:38:29,525 0.6262	0.2792	0.3862	0.2393
2023-03-21 13:38:29,526 
Results:
- F-score (micro) 0.3862
- F-score (macro) 0.3862
- Accuracy 0.2393

By class:
              precision    recall  f1-score   support

     Disease     0.6262    0.2792    0.3862       960

   micro avg     0.6262    0.2792    0.3862       960
   macro avg     0.6262    0.2792    0.3862       960
weighted avg     0.6262    0.2792    0.3862       960

2023-03-21 13:38:29,532 ----------------------------------------------------------------------------------------------------


{'test_score': 0.3861671469740634,
 'dev_score_history': [0.29272419627749574, 0.4144903117101938],
 'train_loss_history': [0.4216455200541671, 0.23894837687515952],
 'dev_loss_history': [0.12372838705778122, 0.08702316880226135]}

In [12]:
SequenceTagger.load("./taggers/lstm-crf/final-model.pt")

2023-03-21 13:38:44,341 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, B-Disease, E-Disease, I-Disease, S-Disease, <START>, <STOP>


SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      '0-pubmed_pmc_wiki_sg_1M.gensim'
      (embedding): Embedding(1000001, 200)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4296, out_features=4296, bias=True)
  (rnn): LSTM(4296, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=8, bias=Tru

In [ ]:
# 2. what label do we want to predict?
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

2023-03-16 04:58:33,906 Computing label dictionary. Progress:


5329it [00:00, 29571.19it/s]

2023-03-16 04:58:34,103 Dictionary created for label 'ner' with 1 values: Disease (seen 5134 times)
Dictionary with 1 tags: Disease


In [ ]:
# 4. initialize fine-tuneable transformer embeddings WITH document context
tf_embeddings = TransformerWordEmbeddings(
    model='emilyalsentzer/Bio_ClinicalBERT',
    layers="-1",
    subtoken_pooling="mean",
    fine_tune=True,
    use_context=True,
)

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tf_tagger = SequenceTagger(
    hidden_size=256,
    embeddings=tf_embeddings,
    tag_dictionary=label_dict,
    tag_type='ner',
    use_crf=True,
    use_rnn=False,
    reproject_embeddings=False,
)

# 6. initialize trainer
tf_trainer = ModelTrainer(tf_tagger, corpus)

2023-03-16 04:58:56,594 SequenceTagger predicts: Dictionary with 5 tags: O, S-Disease, B-Disease, E-Disease, I-Disease


In [ ]:
# 7. run fine-tuning
tf_trainer.fine_tune(
    './taggers/bert-ner',
    learning_rate=5.0e-6,
    mini_batch_size=16,
    max_epochs=1
)

2023-03-16 04:58:59,375 ----------------------------------------------------------------------------------------------------
2023-03-16 04:58:59,378 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28997, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, i

100%|██████████| 57/57 [00:21<00:00,  2.62it/s]

2023-03-16 05:04:49,737 Evaluating as a multi-label problem: False
2023-03-16 05:04:49,762 DEV : loss 0.2649233937263489 - f1-score (micro avg)  0.0885


2023-03-16 05:04:51,119 ----------------------------------------------------------------------------------------------------
2023-03-16 05:04:51,123 Testing using last state of model ...


100%|██████████| 57/57 [00:21<00:00,  2.68it/s]

2023-03-16 05:05:12,412 Evaluating as a multi-label problem: False
2023-03-16 05:05:12,435 0.1121	0.1198	0.1158	0.0615
2023-03-16 05:05:12,436 
Results:
- F-score (micro) 0.1158
- F-score (macro) 0.1158
- Accuracy 0.0615

By class:
              precision    recall  f1-score   support

     Disease     0.1121    0.1198    0.1158       960

   micro avg     0.1121    0.1198    0.1158       960
   macro avg     0.1121    0.1198    0.1158       960
weighted avg     0.1121    0.1198    0.1158       960

2023-03-16 05:05:12,439 ----------------------------------------------------------------------------------------------------


{'test_score': 0.11581067472306145,
 'dev_score_history': [0.08849557522123895],
 'train_loss_history': [0.5786723579456791],
 'dev_loss_history': [0.2649233937263489]}